## Einführung

In dieser Lektion werden folgende Themen behandelt:
- Was ist Function Calling und wofür wird es verwendet
- Wie man einen Function Call mit OpenAI erstellt
- Wie man einen Function Call in eine Anwendung integriert

## Lernziele

Nach Abschluss dieser Lektion wissen und verstehen Sie:

- Den Zweck der Verwendung von Function Calling
- Einrichtung eines Function Calls mit dem OpenAI Service
- Effektive Gestaltung von Function Calls für den Anwendungsfall Ihrer Anwendung


## Verständnis von Funktionsaufrufen

In dieser Lektion möchten wir eine Funktion für unser Bildungs-Startup entwickeln, mit der Nutzer über einen Chatbot technische Kurse finden können. Wir empfehlen Kurse, die zu ihrem Kenntnisstand, ihrer aktuellen Rolle und ihrer bevorzugten Technologie passen.

Dafür nutzen wir eine Kombination aus:
 - `OpenAI`, um eine Chat-Erfahrung für die Nutzer zu schaffen
 - `Microsoft Learn Catalog API`, um Nutzern passende Kurse basierend auf ihrer Anfrage zu zeigen
 - `Function Calling`, um die Anfrage des Nutzers an eine Funktion weiterzuleiten, die dann die API-Abfrage ausführt

Schauen wir uns zunächst an, warum wir überhaupt Funktionsaufrufe verwenden sollten:

print("Messages in next request:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # Eine neue Antwort von GPT erhalten, bei der die Funktionsergebnisse berücksichtigt werden können


print(second_response.choices[0].message)


### Warum Function Calling

Wenn du bereits eine andere Lektion in diesem Kurs abgeschlossen hast, kennst du wahrscheinlich die Leistungsfähigkeit von Large Language Models (LLMs). Hoffentlich hast du aber auch einige ihrer Einschränkungen bemerkt.

Function Calling ist eine Funktion des OpenAI Service, die entwickelt wurde, um folgende Herausforderungen zu lösen:

Inkonsistente Antwortformate:
- Vor Function Calling waren die Antworten eines Large Language Models unstrukturiert und uneinheitlich. Entwickler mussten komplexe Validierungscodes schreiben, um jede Variation im Output zu verarbeiten.

Eingeschränkte Integration externer Daten:
- Vor dieser Funktion war es schwierig, Daten aus anderen Teilen einer Anwendung in einen Chat-Kontext einzubinden.

Durch die Standardisierung der Antwortformate und die nahtlose Integration externer Daten vereinfacht Function Calling die Entwicklung und reduziert den Bedarf an zusätzlicher Validierungslogik.

Nutzer konnten keine Antworten wie „Wie ist das aktuelle Wetter in Stockholm?“ erhalten. Das liegt daran, dass die Modelle auf den Zeitpunkt beschränkt waren, zu dem die Daten trainiert wurden.

Schauen wir uns das folgende Beispiel an, das dieses Problem verdeutlicht:

Angenommen, wir möchten eine Datenbank mit Studentendaten erstellen, um ihnen den passenden Kurs vorzuschlagen. Unten haben wir zwei Beschreibungen von Studierenden, die sich inhaltlich sehr ähnlich sind.


In [ ]:
student_1_description="Emily Johnson is a sophomore majoring in computer science at Duke University. She has a 3.7 GPA. Emily is an active member of the university's Chess Club and Debate Team. She hopes to pursue a career in software engineering after graduating."
 
student_2_description = "Michael Lee is a sophomore majoring in computer science at Stanford University. He has a 3.8 GPA. Michael is known for his programming skills and is an active member of the university's Robotics Club. He hopes to pursue a career in artificial intelligence after finshing his studies."

Wir möchten dies an ein LLM senden, um die Daten zu analysieren. Später kann dies in unserer Anwendung verwendet werden, um es an eine API zu senden oder in einer Datenbank zu speichern.

Lass uns zwei identische Prompts erstellen, mit denen wir das LLM anweisen, welche Informationen uns interessieren:


Wir möchten dies an ein LLM senden, um die für unser Produkt wichtigen Teile zu analysieren. So können wir zwei identische Prompts erstellen, um das LLM anzuleiten:


In [ ]:
prompt1 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_1_description}
'''


prompt2 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_2_description}
'''


Nachdem wir diese beiden Prompts erstellt haben, senden wir sie mit `openai.ChatCompletion` an das LLM. Wir speichern den Prompt in der Variable `messages` und weisen die Rolle `user` zu. Dies dient dazu, eine Nachricht von einem Benutzer an einen Chatbot zu simulieren.


In [ ]:
import os
import json
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()

client = OpenAI()

deployment="gpt-3.5-turbo"

: 

In [ ]:
openai_response1 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt1}]
)
openai_response1.choices[0].message.content 

In [ ]:
openai_response2 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt2}]
)
openai_response2.choices[0].message.content

In [ ]:
# Loading the response as a JSON object
json_response1 = json.loads(openai_response1.choices[0].message.content)
json_response1

In [ ]:
# Loading the response as a JSON object
json_response2 = json.loads(openai_response2.choices[0].message.content )
json_response2

Auch wenn die Prompts gleich sind und die Beschreibungen ähnlich, können wir unterschiedliche Formate für die Eigenschaft `Grades` erhalten.

Wenn du die obenstehende Zelle mehrmals ausführst, kann das Format entweder `3.7` oder `3.7 GPA` sein.

Das liegt daran, dass das LLM unstrukturierte Daten in Form des geschriebenen Prompts entgegennimmt und ebenfalls unstrukturierte Daten zurückgibt. Wir brauchen jedoch ein strukturiertes Format, damit wir wissen, was uns beim Speichern oder Verwenden dieser Daten erwartet.

Durch die Verwendung von funktionalem Aufruf können wir sicherstellen, dass wir strukturierte Daten zurückbekommen. Beim funktionalen Aufruf ruft das LLM tatsächlich keine Funktionen auf oder führt sie aus. Stattdessen erstellen wir eine Struktur, der das LLM bei seinen Antworten folgen soll. Diese strukturierten Antworten nutzen wir dann, um zu wissen, welche Funktion in unseren Anwendungen ausgeführt werden soll.


![Funktionsaufruf-Flussdiagramm](../../../../translated_images/Function-Flow.083875364af4f4bb69bd6f6ed94096a836453183a71cf22388f50310ad6404de.de.png)


### Anwendungsfälle für die Nutzung von Funktionsaufrufen

**Externe Tools aufrufen**  
Chatbots eignen sich hervorragend, um Nutzerfragen zu beantworten. Mithilfe von Funktionsaufrufen können Chatbots Nachrichten von Nutzern nutzen, um bestimmte Aufgaben zu erledigen. Zum Beispiel kann ein*e Student*in den Chatbot bitten: „Sende eine E-Mail an meine Lehrkraft und teile mit, dass ich mehr Unterstützung bei diesem Thema benötige.“ Dies kann einen Funktionsaufruf an `send_email(to: string, body: string)` auslösen.

**API- oder Datenbankabfragen erstellen**  
Nutzer*innen können Informationen in natürlicher Sprache suchen, die dann in eine formatierte Abfrage oder API-Anfrage umgewandelt wird. Ein Beispiel wäre eine Lehrkraft, die fragt: „Welche Schüler*innen haben die letzte Aufgabe abgeschlossen?“ – dies könnte eine Funktion namens `get_completed(student_name: string, assignment: int, current_status: string)` aufrufen.

**Strukturierte Daten erstellen**  
Nutzer*innen können einen Textblock oder eine CSV-Datei nehmen und das LLM nutzen, um wichtige Informationen daraus zu extrahieren. Zum Beispiel kann ein*e Student*in einen Wikipedia-Artikel über Friedensabkommen verwenden, um daraus KI-Lernkarten zu erstellen. Dies kann mit einer Funktion wie `get_important_facts(agreement_name: string, date_signed: string, parties_involved: list)` geschehen.


## 2. Erstellen Ihres ersten Funktionsaufrufs

Der Prozess zum Erstellen eines Funktionsaufrufs umfasst drei Hauptschritte:
1. Aufruf der Chat Completions API mit einer Liste Ihrer Funktionen und einer Benutzernachricht
2. Die Antwort des Modells auslesen, um eine Aktion auszuführen, z. B. eine Funktion oder einen API-Aufruf ausführen
3. Einen weiteren Aufruf an die Chat Completions API machen, diesmal mit der Antwort Ihrer Funktion, um diese Informationen zu nutzen und eine Antwort für den Benutzer zu erstellen.


![Ablauf eines Funktionsaufrufs](../../../../translated_images/LLM-Flow.3285ed8caf4796d7343c02927f52c9d32df59e790f6e440568e2e951f6ffa5fd.de.png)


### Elemente eines Funktionsaufrufs

#### Benutzereingabe

Der erste Schritt ist, eine Benutzernachricht zu erstellen. Diese kann dynamisch zugewiesen werden, indem man den Wert eines Texteingabefelds übernimmt, oder man kann hier einen Wert festlegen. Wenn du zum ersten Mal mit der Chat Completions API arbeitest, müssen wir die `role` und den `content` der Nachricht definieren.

Die `role` kann entweder `system` (Regeln erstellen), `assistant` (das Modell) oder `user` (der Endnutzer) sein. Für Funktionsaufrufe setzen wir dies auf `user` und geben eine Beispiel-Frage an.


In [ ]:
messages= [ {"role": "user", "content": "Find me a good course for a beginner student to learn Azure."} ]

### Funktionen erstellen.

Als Nächstes definieren wir eine Funktion und deren Parameter. Wir verwenden hier nur eine Funktion namens `search_courses`, aber du kannst auch mehrere Funktionen erstellen.

**Wichtig**: Funktionen werden in die Systemnachricht an das LLM aufgenommen und zählen zu den verfügbaren Tokens, die dir zur Verfügung stehen.


In [ ]:
functions = [
   {
      "name":"search_courses",
      "description":"Retrieves courses from the search index based on the parameters provided",
      "parameters":{
         "type":"object",
         "properties":{
            "role":{
               "type":"string",
               "description":"The role of the learner (i.e. developer, data scientist, student, etc.)"
            },
            "product":{
               "type":"string",
               "description":"The product that the lesson is covering (i.e. Azure, Power BI, etc.)"
            },
            "level":{
               "type":"string",
               "description":"The level of experience the learner has prior to taking the course (i.e. beginner, intermediate, advanced)"
            }
         },
         "required":[
            "role"
         ]
      }
   }
]

**Definitionen**

Die Struktur der Funktionsdefinition hat mehrere Ebenen, von denen jede ihre eigenen Eigenschaften besitzt. Hier ist eine Übersicht über die verschachtelte Struktur:

**Eigenschaften der Funktion auf oberster Ebene:**

`name` – Der Name der Funktion, die aufgerufen werden soll.

`description` – Dies ist die Beschreibung, wie die Funktion funktioniert. Hier ist es wichtig, spezifisch und klar zu sein.

`parameters` – Eine Liste von Werten und dem Format, das das Modell in seiner Antwort liefern soll.

**Eigenschaften des Parameters-Objekts:**

`type` – Der Datentyp des Parameters-Objekts (meistens "object")

`properties` – Liste der spezifischen Werte, die das Modell für seine Antwort verwendet

**Eigenschaften einzelner Parameter:**

`name` – Implizit durch den Eigenschaftsschlüssel definiert (z. B. "role", "product", "level")

`type` – Der Datentyp dieses spezifischen Parameters (z. B. "string", "number", "boolean")

`description` – Beschreibung des jeweiligen Parameters

**Optionale Eigenschaften:**

`required` – Ein Array, das auflistet, welche Parameter für den Funktionsaufruf erforderlich sind


### Den Funktionsaufruf durchführen
Nachdem wir eine Funktion definiert haben, müssen wir sie nun im Aufruf der Chat Completion API einbinden. Das machen wir, indem wir `functions` zur Anfrage hinzufügen. In diesem Fall `functions=functions`.

Es gibt außerdem die Möglichkeit, `function_call` auf `auto` zu setzen. Das bedeutet, dass wir dem LLM überlassen, basierend auf der Nutzeranfrage zu entscheiden, welche Funktion aufgerufen werden soll, anstatt dies selbst festzulegen.


In [ ]:
response = client.chat.completions.create(model=deployment, 
                                        messages=messages,
                                        functions=functions, 
                                        function_call="auto") 

print(response.choices[0].message)

Jetzt schauen wir uns die Antwort an und sehen, wie sie formatiert ist:

{
  "role": "assistant",
  "function_call": {
    "name": "search_courses",
    "arguments": "{\n  \"role\": \"student\",\n  \"product\": \"Azure\",\n  \"level\": \"beginner\"\n}"
  }
}

Du siehst, dass der Name der Funktion aufgerufen wird und das LLM anhand der Benutzernachricht die Daten finden konnte, um die Argumente der Funktion auszufüllen.


## 3. Integration von Funktionsaufrufen in eine Anwendung.

Nachdem wir die formatierte Antwort vom LLM getestet haben, können wir diese nun in eine Anwendung integrieren.

### Steuerung des Ablaufs

Um dies in unsere Anwendung zu integrieren, gehen wir wie folgt vor:

Zuerst rufen wir die OpenAI-Services auf und speichern die Nachricht in einer Variable namens `response_message`.


In [ ]:
response_message = response.choices[0].message

Nun werden wir die Funktion definieren, die die Microsoft Learn API aufruft, um eine Liste von Kursen zu erhalten:


In [ ]:
import requests

def search_courses(role, product, level):
    url = "https://learn.microsoft.com/api/catalog/"
    params = {
        "role": role,
        "product": product,
        "level": level
    }
    response = requests.get(url, params=params)
    modules = response.json()["modules"]
    results = []
    for module in modules[:5]:
        title = module["title"]
        url = module["url"]
        results.append({"title": title, "url": url})
    return str(results)



Als bewährte Methode prüfen wir zunächst, ob das Modell eine Funktion aufrufen möchte. Danach erstellen wir eine der verfügbaren Funktionen und ordnen sie der aufgerufenen Funktion zu.  
Anschließend nehmen wir die Argumente der Funktion und ordnen sie den Argumenten aus dem LLM zu.

Zum Schluss fügen wir die Nachricht zum Funktionsaufruf und die Werte hinzu, die von der `search_courses`-Nachricht zurückgegeben wurden. So erhält das LLM alle Informationen, die es benötigt, um dem Nutzer in natürlicher Sprache zu antworten.


In [ ]:
# Check if the model wants to call a function
if response_message.function_call.name:
    print("Recommended Function call:")
    print(response_message.function_call.name)
    print()

    # Call the function. 
    function_name = response_message.function_call.name

    available_functions = {
            "search_courses": search_courses,
    }
    function_to_call = available_functions[function_name] 

    function_args = json.loads(response_message.function_call.arguments)
    function_response = function_to_call(**function_args)

    print("Output of function call:")
    print(function_response)
    print(type(function_response))


    # Add the assistant response and function response to the messages
    messages.append( # adding assistant response to messages
        {
            "role": response_message.role,
            "function_call": {
                "name": function_name,
                "arguments": response_message.function_call.arguments,
            },
            "content": None
        }
    )
    messages.append( # adding function response to messages
        {
            "role": "function",
            "name": function_name,
            "content":function_response,
        }
    )



In [ ]:
print("Messages in next request:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # get a new response from GPT where it can see the function response


print(second_response.choices[0].message)

## Code Challenge 

Super gemacht! Um dein Wissen über OpenAI Function Calling weiter zu vertiefen, kannst du folgendes umsetzen: https://learn.microsoft.com/training/support/catalog-api-developer-reference?WT.mc_id=academic-105485-koreyst 
 - Füge der Funktion weitere Parameter hinzu, die Lernenden helfen könnten, mehr passende Kurse zu finden. Die verfügbaren API-Parameter findest du hier: 
 - Erstelle einen weiteren Funktionsaufruf, der zusätzliche Informationen vom Lernenden abfragt, zum Beispiel die Muttersprache 
 - Baue eine Fehlerbehandlung ein, falls der Funktionsaufruf und/oder der API-Aufruf keine passenden Kurse zurückliefert



---

**Haftungsausschluss**:  
Dieses Dokument wurde mit dem KI-Übersetzungsdienst [Co-op Translator](https://github.com/Azure/co-op-translator) übersetzt. Obwohl wir uns um Genauigkeit bemühen, beachten Sie bitte, dass automatisierte Übersetzungen Fehler oder Ungenauigkeiten enthalten können. Das Originaldokument in seiner Ausgangssprache gilt als maßgebliche Quelle. Für wichtige Informationen wird eine professionelle menschliche Übersetzung empfohlen. Wir übernehmen keine Haftung für Missverständnisse oder Fehlinterpretationen, die sich aus der Nutzung dieser Übersetzung ergeben.
